In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from azure.ai.projects.models import BingGroundingTool,CodeInterpreterTool

from dotenv import load_dotenv
import os
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import BingGroundingTool
from azure.identity import DefaultAzureCredential


# Load environment variables from a .env file
load_dotenv("/home/azureuser/azure-ai-agent-workshop/.env")

In [ ]:
model_client=AzureOpenAIChatCompletionClient(
            model=os.getenv("AZURE_OPENAI_MODEL"),
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_KEY"),
            azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
            api_version=os.getenv("AZUER_OPENAI_API_VERSION")
        )

In [ ]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

In [ ]:
from azure.ai.projects.models import (
    AsyncToolSet,
    BingGroundingTool,
    CodeInterpreterTool
)

toolset = AsyncToolSet()

# Add the code interpreter tool
code_interpreter = CodeInterpreterTool()
toolset.add(code_interpreter)

API_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT")

In [ ]:
async def data_science_ai_agent(query: str) -> str:
    print("This is caculator for Azure AI Agent Service .......")
    
    with project_client:
        agent = project_client.agents.create_agent(
            model=API_DEPLOYMENT_NAME,
            name="Data Science AI Agent",
            instructions="Your role is to assist users with  data science. You are professional about machine learning, AI and so on. You also  can use the code interpreter the validate content when there are some algorithem.",
            toolset=toolset,
            temperature=0.2,
            headers={"x-ms-enable-preview": "true"},
        )
        print(f"Created agent, ID: {agent.id}")

        # Create thread for communication
        thread = project_client.agents.create_thread()
        print(f"Created thread, ID: {thread.id}")

        # Create message to thread
        message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=query,
        )
        print(f"SMS: {message}")
        # Create and process agent run in thread with tools
        run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
        print(f"Run finished with status: {run.status}")

        if run.status == "failed":
            print(f"Run failed: {run.last_error}")

        # Delete the assistant when done
        project_client.agents.delete_agent(agent.id)
        print("Deleted agent")

        # Fetch and log all messages
        messages = project_client.agents.list_messages(thread_id=thread.id)
        print("Messages:"+ messages["data"][0]["content"][0]["text"]["value"])
    return messages["data"][0]["content"][0]["text"]["value"]



In [ ]:
async def save_blog_agent(blog_content: str) -> str:

    print("This is Code Interpreter for Azure AI Agent Service .......")


    code_interpreter = CodeInterpreterTool()
        
    agent = project_client.agents.create_agent(
            model=API_DEPLOYMENT_NAME,
            name="my-agent",
            instructions="You are helpful agent",
            tools=code_interpreter.definitions,
    )

    thread = project_client.agents.create_thread()

    message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content="""
        
                    You are my Python programming assistant. Generate code,save """+ blog_content +
                    
                """    
                    and execute it according to the following requirements

                    1. Save blog content to blog-{YYMMDDHHMMSS}.md

                    2. give me the download this file link
                """,
    )
    # create and execute a run
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
            # Check if you got "Rate limit is exceeded.", then you want to get more quota
        print(f"Run failed: {run.last_error}")

    # print the messages from the agent
    messages = project_client.agents.get_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

        # get the most recent message from the assistant
    last_msg = messages.get_last_text_message_by_sender("assistant")
    if last_msg:
        print(f"Last Message: {last_msg.text.value}")

        # print(f"File: {messages.file_path_annotations}")


    for file_path_annotation in messages.file_path_annotations:

        file_name = os.path.basename(file_path_annotation.text)

        project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./blog")
        

    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")
    return "Saved"

In [ ]:
data_science_agent = AssistantAgent(
    name="data_science_agent",
    model_client=model_client,
    tools=[data_science_ai_agent],
    system_message="You are a search expert, help me use tools to find relevant knowledge",
)

In [ ]:
save_blog_content_agent = AssistantAgent(
    name="save_blog_content_agent",
    model_client=model_client,
    tools=[save_blog_agent],
    system_message="""
        Save blog content. Respond with 'Saved' to when your blog are saved.
    """
)

In [ ]:
write_agent = AssistantAgent(
    name="write_agent",
    model_client=model_client,
    system_message="""
        You are a blog writer, please help me write a blog based on bing search content."
    """
)


In [ ]:
text_termination = TextMentionTermination("Saved")
# Define a termination condition that stops the task after 5 messages.
max_message_termination = MaxMessageTermination(10)
# Combine the termination conditions using the `|`` operator so that the
# task stops when either condition is met.
termination = text_termination | max_message_termination

In [ ]:

reflection_team = RoundRobinGroupChat([data_science_agent, write_agent,save_blog_content_agent], termination_condition=termination)

In [ ]:
await Console(
    reflection_team.run_stream(task="""

                    I am writing a blog about machine learning. Search for the following 3 questions and write a English blog based on the search results ,save it
                    
                    1. What is Machine Learning?
                    2. The difference between AI and ML
                    3. The history of Machine Learning
                               

    """)
)  # Stream the messages to the console.